In this notebook we create the runlist to be used for the error estimation.

In [1]:
import numpy as np
from gammapy.data import DataStore
import yaml

In [2]:
with open("../general_config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

conf = cfg['conf']
for hessera in ['hess1', 'hess2']:
    basedir = f'{cfg["FITS_PROD"]}/{hessera}/std_{conf}_fullEnclosure'
    ds = DataStore.from_dir(basedir, f'hdu-index-bg-latest-fov-radec.fits.gz', f'obs-index-bg-latest-fov-radec.fits.gz')
    table = ds.obs_table

    mask_tel = table['N_TELS'] == cfg['mask_ntels']
    mask_zenpnt = table['ZEN_PNT'] < cfg['mask_zenpnt']
    mask_eventcount = table['EVENT_COUNT'] > cfg['mask_eventcount']
    mask_ontime = table['ONTIME'] > cfg['mask_ontime']
    
    mask_final = mask_tel & mask_zenpnt & mask_eventcount & mask_ontime

    runlist = np.asarray(table['OBS_ID'][mask_final])
    np.savetxt(f'initial_runlist_{hessera}.txt', runlist)